# Bring your own Image embeddings to MySQL AI
MySQL AI provides developers to build rich applications with MySQL leveraging built in machine learning, GenAI, LLMs and semantic search. They can create vectors from documents stored in a local file system. Customers can deploy these AI applications on premise or migrate them to MySQL HeatWave for lower cost, higher performance, richer functionality and latest LLMs with no change to their application. This gives developers the flexibility to build their applications on MySQL EE and then deploy them either on premise or in the cloud.

This notebook demonstrates how to use the MySQL [DISTANCE](https://dev.mysql.com/doc/refman/9.1/en/vector-functions.html) function to compute similarity between image embeddings, enabling meaningful comparison of visual content based on semantic features rather than raw pixel values.

### References
- https://blogs.oracle.com/mysql/post/announcing-mysql-ai
- https://dev.mysql.com/doc/mysql-ai/9.4/en/
- https://dev.mysql.com/doc/dev/mysql-studio/latest/#overview

### Prerequistises

- mysql-connector-python
- pandas 

##### Import Python packages

In [3]:
# import Python packages
import os
import json
import struct
import numpy as np
import pandas as pd
import mysql.connector
from typing import List

### Connect to MySQL AI instance
We create a connection to an active MySQL AI instance using the [MySQL Connector/Python](https://dev.mysql.com/doc/connector-python/en/). We also define an API to execute a SQL query using a cursor, and the result is returned as a Pandas DataFrame. Modify the below variables to point to your MySQL AI instance.

 - In MySQL Studio, connections are restricted to only allow localhost as the host. 
 - In MySQL Studio, the only accepted password values are the string unused or None (authentication happens through the web interface).

In [ ]:
HOST = 'localhost'
PORT = 3306
USER = 'root'
PASSWORD = 'unused'
DATABASE = 'mlcorpus'


myconn = mysql.connector.connect(
    host=HOST,
    port=PORT,
    user=USER,
    password=PASSWORD,
    database=DATABASE,
    allow_local_infile=True,
    use_pure=True,
    autocommit=True,
)
mycursor = myconn.cursor()


In [0]:
# Helper function to execute SQL queries and return the results as a Pandas DataFrame
def execute_sql(sql: str) -> pd.DataFrame:
    mycursor.execute(sql)
    return pd.DataFrame(mycursor.fetchall(), columns=mycursor.column_names)

# Bring your own image embeddings

An embedding is a high-dimensional vector that represents the semantic meaning of data. 

Structurally similar or conceptually related inputs map to nearby points in embedding space. In the case of images, visually or semantically similar images yield embeddings that are “close” (high similarity) to each other.

In this notebook, we have computed image embeddings using a transformer-based model called [CLIP](https://openai.com/index/clip/) (Contrastive Language–Image Pretraining), developed by OpenAI. CLIP is trained on a large dataset of image–text pairs to learn a shared embedding space where semantically related images and texts are positioned close together.


In [0]:
dog1_image_embedding = [0.035017,0.019141,-0.013487,0.004338,-0.002529,-0.065731,0.021204,0.049277,0.019692,0.036616,-0.054276,-0.014386,0.075956,0.019661,0.064106,0.008167,0.091646,-0.009034,0.014821,-0.013166,-0.052938,0.002347,0.038918,-0.062864,-0.004930,0.034834,0.028797,-0.005334,-0.004359,-0.014176,0.014523,0.029935,0.000178,-0.006111,-0.015985,0.023429,0.046238,0.030029,0.003484,0.039323,-0.051585,0.010443,-0.009600,-0.008422,0.011538,-0.026712,-0.017631,-0.006503,0.030585,-0.002862,-0.000861,0.014540,0.026815,0.008882,0.021079,0.027975,0.014155,-0.008076,-0.044949,0.004789,0.123476,-0.006307,0.030249,0.040366,-0.012477,-0.009417,-0.014521,0.040303,0.003963,-0.000585,-0.019625,-0.011950,-0.011595,-0.003059,-0.030641,0.017839,-0.004185,-0.019464,0.009679,-0.020502,0.012296,-0.004197,-0.015192,-0.066219,0.041159,-0.030243,0.029019,-0.004065,-0.001433,0.005950,-0.007170,-0.037223,-0.699751,0.024826,0.012656,-0.012611,-0.019255,0.000830,-0.058757,0.044192,0.018640,-0.028177,0.045755,0.026778,-0.000820,0.005558,0.001371,0.012431,0.010161,-0.004316,0.012941,0.040309,0.034155,-0.046231,-0.018259,-0.017214,-0.026657,-0.004087,-0.016059,-0.014599,0.007117,0.085803,0.043225,0.005842,0.021401,-0.003759,-0.006672,0.031420,0.022420,-0.050322,0.034072,0.001068,-0.049254,0.085216,0.018939,0.015600,0.008012,-0.056257,-0.011534,0.018735,-0.018391,0.020567,-0.033392,0.027923,-0.058647,0.057524,-0.014195,0.038774,-0.039489,0.017512,0.005985,0.017253,0.043457,-0.016714,-0.047068,0.022669,-0.002131,-0.037320,-0.039423,-0.008092,0.003643,0.008607,0.001309,-0.041271,0.058193,0.012271,0.003900,-0.022296,0.011247,0.046761,-0.033576,0.000965,0.017626,-0.011965,0.008191,0.024010,0.015013,-0.016240,0.088183,-0.007887,-0.014961,0.012439,0.011727,-0.002776,0.015607,0.012405,-0.035009,-0.007837,-0.011091,-0.006906,0.011935,-0.008301,0.013103,-0.020396,-0.002078,0.020603,0.002364,-0.019678,-0.104223,0.033914,0.027218,-0.007983,0.022759,0.014779,-0.029925,-0.040197,-0.026255,-0.042910,0.009665,0.043513,0.037882,0.037284,0.018288,0.046022,-0.009836,-0.042232,-0.005241,0.039640,0.122965,0.008040,0.059053,0.036045,0.010988,0.009133,-0.028831,-0.028581,0.014857,-0.005747,0.003520,-0.009889,0.028871,0.032320,0.006763,0.010174,0.037285,0.001981,-0.001563,-0.074111,0.033648,-0.016506,0.006863,0.030824,0.005913,-0.003697,0.024574,0.064399,0.032118,-0.033698,0.002501,0.017557,0.015210,0.002553,0.043472,-0.008870,0.019047,0.000811,0.038212,0.008641,-0.038752,-0.008751,-0.044203,0.008043,0.034548,-0.000533,-0.023985,0.002187,-0.025736,-0.019555,-0.041558,0.032676,0.000866,0.027394,0.047261,0.008379,0.016901,0.023255,0.007631,0.007276,-0.010641,0.037068,-0.018152,-0.043177,-0.043891,-0.019778,-0.025094,0.016852,-0.042344,-0.010159,0.008557,-0.000097,0.011001,0.051471,-0.011610,0.034946,0.006734,-0.004552,-0.012068,0.008420,0.033071,-0.001734,-0.028499,0.007058,-0.046853,0.008529,0.020905,0.007794,0.009505,0.031282,0.018618,0.012989,-0.003440,0.029389,0.085242,-0.039708,0.013013,-0.012117,0.025372,0.016684,-0.068441,-0.042979,0.066589,0.069161,-0.027693,0.050357,0.013638,0.009899,-0.022994,-0.008216,0.005204,0.034814,0.020574,-0.026033,-0.001493,-0.038233,0.006859,0.035878,0.035533,0.044259,-0.033538,-0.027591,-0.008794,-0.030026,-0.014715,-0.006605,0.018121,0.030673,0.024370,0.019673,-0.014985,-0.049013,-0.048170,0.016791,-0.000982,-0.023444,0.019791,0.052282,-0.003144,-0.103422,0.020130,-0.015071,0.056515,-0.042557,-0.005268,0.010772,-0.079677,0.000072,-0.037555,-0.017562,-0.034329,-0.035104,-0.013718,-0.010100,-0.022255,0.020267,-0.040480,-0.021042,0.077327,0.014085,0.003404,0.026545,0.055386,-0.030558,-0.008112,-0.025873,0.026707,-0.036744,-0.036358,-0.020917,0.040458,-0.077249,-0.034862,-0.085602,0.017054,0.003105,0.023881,-0.013710,-0.009984,-0.063184,-0.077381,-0.075028,0.002266,-0.031682,-0.001774,-0.017988,0.001740,-0.023218,-0.013029,-0.034750,0.051632,-0.044810,0.068225,-0.067769,0.005670,0.008818,-0.018754,-0.010797,-0.002932,0.023626,0.014921,-0.031375,0.021887,-0.032561,-0.011375,-0.025455,0.000392,-0.042678,0.006043,0.010818,-0.069437,-0.013110,-0.008541,0.012058,-0.003356,-0.005466,-0.010199,0.007490,-0.045322,0.014056,-0.010172,-0.001507,-0.002952,0.064321,-0.051917,0.002333,0.023312,0.008509,-0.008051,0.010864,-0.045859,-0.002523,0.007577,-0.030701,0.061075,0.041604,-0.010872,0.000257,0.010691,0.023341,-0.003621,0.019159,-0.000270,0.039450,0.012771,0.006616,0.091880,-0.011539,-0.047493,0.021492,-0.034012,0.003073,0.052094,0.011922,-0.013475,0.000865,0.000886,0.010507,-0.025364,0.006206,-0.002073,-0.027149,0.018659,-0.028258,0.014152,-0.033591,-0.024771,0.019720,-0.010016,-0.028875,-0.015944,0.012888,-0.017273,-0.018035,0.049587,-0.019404,0.001062,-0.025971,-0.005638,0.034168,-0.041170,-0.001066,-0.020940,0.003212,0.005190,0.016791,-0.041927,0.013478,-0.032876,-0.029698,-0.054168,0.019219,0.015504,-0.012964]
dog2_image_embedding = [0.026360,0.009627,-0.022415,0.000724,-0.008058,-0.067460,0.015470,0.039297,0.003044,0.025826,-0.064096,-0.007970,0.067523,0.016063,0.069946,0.008301,0.090895,0.012169,0.011719,0.002631,-0.031068,0.012372,0.036321,-0.047695,-0.023066,0.016323,0.031246,0.000783,0.013475,-0.022597,0.037835,0.025654,-0.009124,0.009698,-0.012599,0.022895,0.046008,0.040800,-0.008640,0.033431,-0.036940,0.021853,-0.022718,-0.009348,0.015759,0.021942,-0.006194,-0.017008,0.020255,0.010062,0.004835,0.012782,0.020365,-0.010136,0.034565,0.024601,-0.003288,0.006183,-0.034328,0.005165,0.131398,-0.001922,0.023063,0.025590,-0.023732,-0.014520,-0.022815,0.049837,-0.015812,0.011807,-0.022390,0.001672,-0.003552,0.007110,-0.054895,-0.017165,-0.000448,-0.022488,0.015896,-0.011305,0.020854,-0.010586,-0.015166,-0.052981,0.038481,-0.035442,0.058959,-0.000616,0.037292,0.011357,-0.028472,-0.033681,-0.693960,0.025852,0.001832,-0.008961,-0.021345,0.012390,-0.071794,0.044125,0.006247,-0.009752,0.035538,0.009371,0.010029,-0.005546,0.025671,0.015592,0.019818,-0.006713,0.007475,0.037606,0.046924,-0.046573,-0.003103,0.007815,-0.024506,-0.016373,-0.014714,-0.002677,-0.003683,0.097676,0.047998,-0.003796,-0.000877,-0.019569,-0.005856,0.037434,0.014057,-0.062564,0.018910,0.011928,-0.050075,0.086028,0.003828,0.006057,0.024096,-0.046660,-0.005294,-0.000298,0.002501,0.019295,-0.061324,0.027771,-0.053517,0.055738,0.013239,0.041317,-0.021391,0.037300,-0.020163,0.016302,0.033827,-0.014724,-0.046168,0.028401,-0.008673,-0.053509,-0.059522,0.014970,-0.000003,0.020634,0.008000,-0.039556,0.053464,0.009412,-0.003683,-0.020901,0.002691,0.062037,-0.017135,0.005138,0.025125,-0.007522,0.009386,0.026791,0.026169,-0.018460,0.090391,0.000271,-0.026331,0.010473,0.030387,-0.008331,0.000929,0.003431,-0.055314,-0.001718,-0.006331,0.003417,0.016463,-0.002066,0.008487,-0.023215,0.002280,0.033159,-0.006019,-0.000690,-0.105339,0.019716,0.026007,-0.001536,0.036072,0.015518,-0.032323,-0.036362,-0.022415,-0.032050,0.007683,0.040976,0.028467,0.025402,0.017728,0.044338,0.001078,-0.029761,-0.021600,0.029537,0.119872,0.003593,0.072864,0.041245,0.022775,0.027961,-0.025230,-0.027204,0.021609,-0.012819,-0.001177,-0.006057,0.011972,0.035565,0.005027,0.018731,0.030220,-0.017034,-0.004598,-0.069186,0.044536,-0.007664,0.016001,0.040979,0.013234,-0.012830,0.028257,0.043437,0.039496,-0.043502,0.019244,0.008284,0.022009,-0.004514,0.033436,-0.024086,0.010675,0.009378,0.041782,0.000038,-0.021445,-0.022015,-0.059115,0.017813,0.032739,-0.019376,-0.040016,0.006085,-0.017778,-0.030710,-0.013770,0.021562,-0.013246,0.020682,0.044449,0.011830,0.006341,0.019097,0.012131,0.033558,-0.013165,0.030878,-0.015950,-0.031284,-0.056384,-0.013418,-0.024376,0.012582,-0.046293,0.010264,0.013515,0.028473,0.014862,0.042744,-0.020309,0.043502,0.009881,-0.013375,0.004180,0.019937,0.015816,-0.027633,-0.022927,-0.008963,-0.043425,-0.005544,0.025132,0.030452,0.024801,0.013603,0.010405,0.039800,-0.008141,0.024438,0.086053,-0.048144,0.009021,-0.028575,0.022558,0.018215,-0.057516,-0.036139,0.070221,0.077746,-0.014289,0.049325,0.028744,0.004874,-0.013275,0.018219,0.016818,0.038277,0.021636,-0.029680,-0.022781,-0.030778,0.006555,0.007952,0.051724,0.020867,-0.021438,-0.039176,-0.010672,-0.019755,-0.026397,-0.011172,0.017840,0.023975,0.013811,0.027768,-0.016458,-0.038463,-0.051072,0.007821,0.000022,-0.009709,0.012745,0.045097,0.003713,-0.081659,0.025950,-0.011162,0.043636,-0.035052,0.007059,0.025219,-0.076499,0.015599,-0.039615,-0.000609,-0.042712,-0.032744,-0.026059,-0.002962,-0.023742,0.017420,-0.047394,-0.018445,0.085539,-0.001277,-0.001904,0.019434,0.057870,-0.031467,0.003640,-0.018590,0.024489,-0.032924,-0.043226,0.010206,0.051624,-0.098476,-0.005850,-0.092577,0.017867,0.005293,0.013030,0.000515,0.002290,-0.075327,-0.087682,-0.066864,-0.004409,-0.012404,-0.009777,-0.010008,-0.007742,-0.006426,-0.014540,-0.016951,0.046802,-0.039388,0.066587,-0.061043,0.005610,0.018023,-0.013818,-0.000416,-0.001447,0.029477,0.009614,-0.021177,0.027719,-0.016205,-0.013989,-0.005077,0.006991,-0.023527,0.006496,-0.001448,-0.094913,-0.006113,0.008220,0.017783,-0.007588,-0.001756,-0.015512,-0.009482,-0.052080,0.009049,0.003948,0.002506,-0.003985,0.050292,-0.020474,-0.011434,0.026200,0.007290,-0.001724,0.007108,-0.024536,-0.014048,-0.000147,-0.039910,0.056855,0.038188,-0.018061,0.006656,0.003246,0.024230,0.004779,0.020647,0.005803,0.019608,0.013768,-0.011047,0.064894,-0.006042,-0.032886,0.025396,-0.040078,0.001020,0.074353,0.006646,-0.007235,0.004398,0.004983,0.004241,-0.021339,-0.016241,0.013891,-0.019236,0.023355,-0.020302,0.000087,-0.044975,-0.033852,0.003637,0.000056,-0.027838,-0.027992,0.014818,-0.023972,-0.020310,0.040538,-0.003344,0.018044,-0.025569,-0.007443,0.031715,-0.041389,-0.010463,-0.016506,-0.001658,0.022670,-0.008485,-0.047886,0.026301,-0.046795,-0.036778,-0.036175,0.024129,0.009188,-0.006446]
cat_image_embedding = [-0.006186,-0.037262,0.002448,-0.042662,-0.014037,-0.026032,0.017309,0.033944,-0.000154,-0.009503,-0.007272,-0.021885,0.050052,-0.019645,0.062476,-0.006100,0.042589,0.004129,0.067607,0.045207,-0.040958,0.037365,0.068021,-0.035824,-0.083865,-0.007413,0.047700,0.012467,-0.004309,0.023912,0.002308,0.005179,-0.004617,0.025577,0.022216,0.020756,0.028323,0.012220,-0.004302,0.119414,-0.025892,-0.042321,0.000417,-0.003370,0.006376,-0.057198,-0.030998,-0.005715,0.031140,-0.034868,0.017242,0.018598,0.014019,-0.012740,0.017232,-0.003547,0.014975,0.080045,-0.006870,-0.031954,0.129879,-0.011094,-0.021025,-0.013478,-0.018144,-0.043897,0.030691,0.032044,-0.005491,0.026144,-0.020480,0.000745,-0.011977,0.008509,-0.058874,-0.043237,0.011462,-0.040206,-0.013892,0.008120,-0.019240,-0.049260,-0.028393,-0.024091,0.004141,0.005921,0.068496,-0.020801,0.014502,0.006174,0.004497,-0.040623,-0.682446,0.037915,-0.004830,-0.011082,-0.008843,-0.029873,-0.127821,0.060517,0.000976,-0.012873,0.002327,0.026167,0.013957,0.007025,0.002067,0.040458,-0.033264,-0.036378,0.017704,-0.025332,0.042769,0.008631,0.008382,0.025077,-0.020555,0.019120,0.027612,-0.003889,0.022098,0.037956,0.028127,-0.032894,0.023985,-0.019305,-0.025273,0.033162,-0.024484,-0.015474,0.007200,-0.013259,-0.025131,0.080562,-0.034529,-0.001328,-0.004690,0.000951,0.019616,-0.036275,0.036482,0.017622,-0.006520,0.046339,-0.048169,-0.002403,0.003517,0.018887,0.019029,0.056132,-0.013714,0.011193,0.010635,-0.030963,-0.013136,0.008099,-0.041711,-0.040813,-0.035311,0.001278,0.016136,0.010478,-0.025266,-0.027006,0.042094,-0.000626,0.022047,0.033495,-0.011546,0.045621,-0.012632,0.014967,0.013135,0.021752,-0.009822,-0.007300,0.005808,-0.012655,0.028557,0.021780,-0.014277,-0.011111,0.000819,0.007732,-0.030575,0.008680,-0.013792,0.013235,0.009106,-0.013790,0.015657,0.007949,0.004740,-0.035411,-0.021628,0.003084,0.001561,-0.013138,-0.088364,-0.039874,0.001570,-0.006006,-0.001543,0.023786,-0.011873,-0.004353,0.015798,0.006195,-0.022837,0.032732,-0.007158,0.020824,-0.030725,0.025701,-0.031644,-0.029365,-0.018595,-0.000873,0.131689,-0.037940,0.069883,0.040232,0.016051,0.001434,-0.001210,0.017281,0.020466,-0.016180,-0.007531,0.002988,-0.001405,0.010748,-0.007860,0.071897,0.020962,-0.006979,0.015323,-0.008051,0.047840,-0.031894,0.035520,0.004695,0.013307,0.012373,0.052133,0.026521,0.001261,-0.068218,0.010300,-0.004165,-0.021886,0.002892,0.006145,-0.008053,0.007331,0.028758,-0.016527,-0.012612,-0.028198,-0.022418,-0.060392,0.014736,0.002402,-0.049944,0.003750,-0.042497,-0.033717,-0.017998,-0.031179,0.027974,-0.019257,-0.011825,0.022400,0.005167,-0.025582,0.004511,-0.020716,0.042736,-0.025135,-0.008232,0.008504,0.017256,-0.028641,0.018185,-0.004009,0.009852,-0.075212,0.007968,-0.008148,-0.003897,-0.064115,0.003353,0.020029,0.003441,0.011014,-0.011452,-0.025419,-0.003722,0.004871,0.015321,0.018808,-0.003668,0.003237,-0.020395,-0.031583,0.004119,0.016819,0.002120,0.056676,0.015839,-0.013595,0.012927,0.080437,-0.041991,0.003770,0.037805,-0.018016,0.028724,-0.040228,-0.035823,0.077886,0.067495,-0.028922,0.022227,0.027888,-0.006334,-0.009957,0.035941,0.032463,0.021090,0.003304,-0.056177,0.012944,0.009928,0.025539,-0.007650,0.056904,0.009480,0.022757,-0.036383,-0.016183,-0.037751,0.007587,0.041488,-0.028256,-0.063955,0.004398,0.005805,0.010513,-0.013759,-0.069847,-0.014129,0.057684,-0.011347,-0.019952,-0.008434,0.034737,-0.000572,-0.013439,0.008708,-0.030241,0.022654,0.051385,0.036412,-0.062074,-0.028245,-0.014838,-0.008148,-0.025090,-0.003792,0.015845,0.035407,0.004672,-0.022917,-0.054851,-0.032424,0.173446,0.014606,0.010560,0.031653,0.049426,-0.075302,-0.033093,-0.004828,0.054900,-0.001449,-0.026518,-0.033615,-0.014316,-0.033832,0.026330,-0.056190,0.038261,0.017704,-0.015407,0.017697,-0.010647,-0.034036,-0.064847,-0.014079,-0.034981,0.020554,-0.002215,0.006086,-0.027123,0.008178,-0.023359,-0.032954,0.071352,0.002620,0.061944,0.000191,-0.043562,-0.018520,-0.005276,-0.013081,0.023722,0.000616,0.010755,-0.041009,-0.025440,0.037934,0.021877,-0.038530,0.030784,0.033450,-0.001600,-0.038628,-0.085764,0.020216,0.033003,0.021216,0.029358,0.005139,0.027252,-0.037800,-0.042542,0.038996,0.037732,-0.029292,0.009002,0.032935,-0.002300,-0.026356,0.035736,0.008103,0.013043,0.014909,-0.023062,-0.038019,0.014084,-0.031934,-0.003412,0.040161,-0.056136,-0.021067,0.051637,0.015997,0.010169,0.027396,0.003975,-0.021315,-0.039075,0.004452,0.036004,0.025492,-0.018562,-0.040460,0.001351,-0.007045,0.053834,-0.006391,-0.033819,0.003145,-0.038685,-0.006489,-0.028226,-0.027501,0.051580,-0.036524,0.022272,-0.033454,-0.005147,-0.015983,-0.059782,0.036183,0.018492,-0.003666,-0.058939,-0.021475,0.029923,-0.001166,0.049644,-0.024735,0.027871,0.039822,-0.002080,0.031018,-0.045288,0.003545,-0.021754,0.006920,0.000291,-0.029558,0.012034,0.001303,-0.012126,0.006984,0.034920,0.040821,-0.018394,0.006306]
street_image_embedding = [-0.009287,0.012144,0.021285,0.018630,0.022018,-0.037264,0.025607,0.027092,0.057012,-0.011252,-0.007282,-0.039937,-0.043867,0.018018,0.006401,-0.011508,-0.131914,0.006002,-0.001251,-0.040710,-0.018353,-0.009965,-0.006011,-0.016728,-0.013722,0.010354,-0.043714,-0.017408,-0.035890,-0.007078,-0.023314,0.042666,-0.026184,-0.067461,0.105681,-0.037562,0.005738,0.025874,0.001780,0.119674,0.021510,0.009246,-0.019430,-0.014091,0.007034,-0.021742,0.027265,-0.020670,0.010777,0.028957,0.025819,-0.036009,0.048066,-0.031634,-0.030702,-0.000789,-0.017240,0.041665,0.063855,0.036608,-0.134370,-0.004327,-0.006352,-0.004488,-0.000859,-0.009744,0.019396,0.069337,-0.013308,-0.002061,-0.003393,0.003618,0.018500,-0.022047,0.006122,0.007194,0.007102,0.043830,-0.009691,-0.045506,-0.006053,0.010784,0.022660,-0.016236,0.022789,0.032602,0.070919,-0.007175,-0.063035,-0.026210,0.023206,-0.021376,-0.609646,0.051037,-0.017603,0.027253,0.006694,0.041685,-0.030516,-0.009552,0.031258,-0.014762,-0.050391,-0.020209,0.009326,0.031852,-0.148658,0.016018,-0.027980,-0.010121,0.015480,-0.103834,0.019642,-0.018313,0.021494,-0.001032,0.004657,0.013690,0.003963,-0.009771,0.001940,-0.048042,0.006084,-0.032234,-0.012098,-0.026332,0.002214,0.009811,-0.009413,0.020168,-0.008475,0.033867,-0.010014,0.080446,0.002153,0.024088,0.019840,-0.034648,0.039493,0.028316,0.020577,0.019143,-0.023163,0.030088,-0.029660,-0.014834,-0.032802,0.054243,-0.043797,-0.013376,0.035152,-0.014551,0.122235,-0.001230,-0.011618,0.003101,-0.022259,0.018002,-0.006795,-0.007930,-0.027481,-0.021515,-0.002819,0.008923,0.025576,-0.018843,0.046666,0.036025,0.028940,-0.036752,0.002162,0.007285,-0.025159,0.010791,0.016641,0.073315,0.062319,0.018460,0.017523,0.025049,0.010620,-0.053222,0.004893,0.004919,-0.032036,0.011412,-0.006567,0.021682,-0.008364,0.019554,-0.001414,-0.013076,-0.003388,0.063803,-0.032959,-0.033477,-0.010286,-0.031959,-0.024844,0.009687,0.007222,0.044013,0.003450,0.024660,0.006417,-0.014386,0.018276,-0.010477,0.001869,0.018092,0.008492,0.081968,0.003977,-0.030670,-0.009133,-0.041209,0.047878,-0.018639,-0.024909,0.007699,0.008951,0.038165,0.003751,0.014336,-0.017286,-0.024476,0.015115,-0.021310,0.017954,0.017144,0.009820,-0.013892,0.009252,-0.029850,-0.000943,0.023908,0.051237,-0.037014,-0.032757,0.038935,-0.004739,0.000794,0.017805,-0.035244,0.018831,0.019815,0.044019,0.016480,0.011138,-0.001674,-0.031812,0.007415,0.020266,0.013040,0.003490,-0.002075,-0.054529,-0.004004,0.065482,-0.000787,0.021252,0.050473,-0.007622,0.026577,0.008382,-0.017490,0.003844,0.040437,-0.017757,-0.004555,-0.022427,0.022827,0.001404,0.018724,-0.023147,0.021074,-0.035152,-0.014440,0.015660,-0.036639,-0.007709,-0.009261,-0.051949,0.004357,0.010623,-0.026180,-0.212481,-0.029972,-0.003978,-0.010126,-0.008125,0.002018,-0.051672,0.041732,0.005820,0.008329,-0.090780,0.010346,-0.006250,0.039582,0.025095,0.005482,-0.052904,-0.008297,0.031526,-0.044153,0.022645,0.030653,0.028864,0.043391,0.024662,-0.009229,0.080547,0.027455,-0.023574,-0.024304,0.034688,-0.015826,0.010517,0.034272,0.054192,-0.018250,-0.023379,-0.006996,-0.000698,0.009779,-0.017735,0.002769,0.009510,-0.030256,0.025123,-0.016574,-0.001338,-0.024760,0.003175,0.002305,0.052413,0.027993,0.017319,0.012624,-0.012296,-0.021911,0.022456,0.007877,-0.016784,-0.017256,-0.040574,0.031742,-0.030087,-0.038945,0.003514,-0.033392,0.033963,-0.003745,0.025425,-0.047220,-0.069117,0.116369,0.039342,-0.012053,-0.019448,-0.017067,0.029816,-0.043134,0.115392,0.016472,-0.023279,0.098036,0.023185,-0.001059,-0.004127,0.014300,-0.017195,-0.037629,0.008712,0.019164,-0.007072,-0.033940,0.014126,0.001955,0.021607,0.005108,-0.004430,-0.018656,-0.001631,-0.013168,-0.007712,-0.030834,0.000754,-0.174452,0.009702,-0.021341,0.056320,-0.004590,-0.027273,-0.040145,-0.050527,0.001642,0.120550,-0.044401,0.007344,0.020587,0.024396,0.013542,-0.041620,0.011686,-0.015930,0.031433,0.052435,0.036297,-0.010660,0.043708,0.001641,-0.009400,-0.013012,-0.046169,-0.017189,-0.058452,0.004251,-0.037902,0.012005,-0.013990,0.040257,0.026579,-0.024986,0.030004,0.003756,0.012632,-0.016456,0.008323,0.000418,0.020143,-0.023343,0.004193,-0.038559,0.019557,-0.034544,0.028778,0.015963,-0.036183,-0.014330,0.000163,0.030124,0.014324,-0.003855,0.010064,0.006491,-0.001036,0.016050,-0.019837,-0.037041,0.003691,-0.016200,0.106679,0.002019,0.036581,0.073188,-0.048561,-0.004535,-0.048044,-0.041549,-0.001637,0.053909,-0.056168,0.046249,-0.018862,-0.029520,-0.002222,-0.014341,0.018470,-0.017471,0.022745,-0.010209,-0.034453,0.001131,-0.002561,0.003785,-0.018047,-0.048316,-0.044291,0.006570,-0.052165,-0.003650,-0.003568,0.004901,0.018199,-0.037824,0.007389,-0.031771,-0.054532,0.002253,-0.013882,0.017743,-0.014075,-0.028646,-0.020095,0.023269,0.020613,0.014348,0.018897,-0.042991,0.001801,-0.015664,0.014374,0.083486,-0.005203,-0.020075,0.037131,-0.012789,0.051308,-0.026965,-0.012707]

# Comparing image embeddings

Once image embeddings are available, we can **quantify similarity** between any two images by computing a score. In this notebook, we compute similarity using MySQL’s [DISTANCE](https://dev.mysql.com/doc/refman/9.1/en/vector-functions.html) function with the **"DOT"** metric (i.e. the dot product).

Thus, a higher similarity score indicates greater alignment (i.e. more similar images) under the embedding space.  
You can use these scores to **rank** image pairs, retrieve nearest neighbors, or filter by similarity thresholds.

---

Below are illustrative cases showing how similarity scores between image embeddings tend to behave.

### Case 1: Identical images
When two images are exactly the same (pixel-for-pixel or with trivial metadata differences), their embeddings should be nearly identical. Thus, the similarity score should be **very close to 1.0**.


In [0]:
df = execute_sql(f"""SELECT DISTANCE(STRING_TO_VECTOR("{dog1_image_embedding}"), STRING_TO_VECTOR("{dog1_image_embedding}"), "DOT");""")
float(df.iat[0,0])


1.0000005960464478

### Case2: Different images of the same class / object type
For example, two different photographs of **dogs** (not the same dog). Because they share strong semantic and visual features, they should yield a **high similarity score**, though lower than Case 1.


In [0]:
df = execute_sql(f"""SELECT DISTANCE(STRING_TO_VECTOR("{dog1_image_embedding}"), STRING_TO_VECTOR("{dog2_image_embedding}"), "DOT");""")
float(df.iat[0,0])


0.9645994305610656

### Case 3: Related but different categories / content
Consider images of a **dog** versus a **cat**, or two animals of different species. These share some semantic traits (e.g. “animal”, “fur”, “face”), so the score is expected to be **moderate** — not extremely high, but significantly above random noise.


In [0]:
df = execute_sql(f"""SELECT DISTANCE(STRING_TO_VECTOR("{dog1_image_embedding}"), STRING_TO_VECTOR("{cat_image_embedding}"), "DOT");""")
float(df.iat[0,0])


0.7443138360977173

### Case 4: Semantically unrelated images
For example, an image of a **dog** versus an image of a **street scene**. These share little in the embedding space, so their similarity score should be **low**, close to the lower bound.


In [0]:
df = execute_sql(f"""SELECT DISTANCE(STRING_TO_VECTOR("{dog1_image_embedding}"), STRING_TO_VECTOR("{street_image_embedding}"), "DOT");""")
float(df.iat[0,0])


0.449489563703537

The similarity score decreases as the semantic difference between the images increases.

Close the MySQL connection

In [0]:
mycursor.close()  # Closes the cursor
myconn.close()    # Closes the connection

We invite you to try [HeatWave AutoML and GenAI](https://www.oracle.com/heatwave/free/). If you’re new to Oracle Cloud Infrastructure, try Oracle Cloud Free Trial, a free 30-day trial with US$300 in credits.